In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from groq import Groq
from src.llm_engine.gemini_agent import Agent


In [3]:
import pandas as pd
df = pd.read_csv(r"./data/GridPuzzle_processed.csv")
df = df[df["id"].isin([13, 821, 687, 718, 350, 1084, 1677, 1437, 1597, 1935, 1020, 1495, 2893, 2208, 2700, 2788, 3720, 3936, 3642, 3659, 4465, 4658, 4087, 4861, 4140, 4464, 4848, 3954, 5740, 5675, 5212, 4976, 5889, 5659, 5325, 5585, 6843, 6292, 6045, 6432, 6823, 5923, 6778, 6997, 7685, 7865, 7183, 7389, 7866, 7188, 7837, 8712, 7959, 8356, 8142, 7993, 8519, 9106, 9719, 9139, 9487, 8853, 8988, 9043, 9497, 9596, 9965, 10037, 10380, 10100, 10450, 10440, 9976, 10251, 11592, 11097, 11022, 11603, 11546, 11748, 11263, 11367, 11407, 11304, 11321, 11923, 12703, 12356, 12535, 12172, 12133, 12395, 12818, 13003, 12900, 13407, 13198, 12782, 13051, 12980])]
df.head(1)

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
6,341.json,13,A group of friends has decided to try several ...,3 | Raymond | vegan\n5 | Wesley | gluten-free\...,|------------|------------|\n | 3 | Raymond |\...,Let's solve the puzzle step by step!\n \n Step...,"Step-by-step solution:\n 1. From clue 3, we kn...","Step-by-step solution:\n \n 1. From Clue 3, we...","Step-by-step solution:\n 1. From clue 3, we kn...",A group of friends has decided to try several ...,Clues:\n1. Stuart lost 4 more pounds than the ...,"pounds lost : 3, 5, 7, 9.\nnames : Raymond, St...",5,3,lost names diets


In [3]:
client = Groq(
    api_key= os.getenv("GROQ_API_KEY"),
)

In [3]:
import re
def process_ls(ls):
    if re.search(r"\d+",ls[0]) is None:
        return ls

    else:
        res = []
        for val in ls:
            val = re.sub(r"[^\d+]", '', val)
            res.append(val)
        return res


In [4]:
import re
def process_question(question):
    lines = question.split("\n")
    categories = {}
    clues = []
    actual_question = ""
    ckpt = -1
    flag = -1
    for index, line in enumerate(lines):


        if ckpt == 1 and re.search(r"^\d+\.", line) is None:
            break

        if " : " in line:
            if flag == -1:
                flag = index
            title, ls = line.split(" : ")[0], line.split(" : ")[1].split(", ")
            ls = process_ls(ls)
            categories[title] = ls
        
        if "Clues" in line:
            ckpt = 1
            continue
        
        if ckpt == 1 and re.search(r"\d+\.", line) is not None:
            clues.append(line)
        


    actual_question += "\n".join(lines[:flag])            

    return actual_question, categories, clues


#ques, categories, clues = process_question(df["question"].iloc[100])


In [6]:
path = r"./deepseek_responses"
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
for i, row in df.iterrows():
    
    id_ = row["id"]
    user_query = row["question"]


    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{
                "role": "user",
                "content": user_query

            }],
        temperature=0,
        max_completion_tokens=8192,
        top_p=1,
        stream=True,
        stop=None,
    )

    groq_deepseek_response = ""
    for chunk in completion:
        text = chunk.choices[0].delta.content or ""
        groq_deepseek_response += text

    groq_deepseek_response


    with open(os.path.join(path, f"deepseek_pred_{id_}.txt"), "w") as f:
        f.write(groq_deepseek_response)

In [4]:
path = r"./gemini_responses"
if not os.path.exists(path):
    os.mkdir(path)

In [12]:
ids= []
for file in os.listdir(path):

    id_ = file.split("_")[2].split(".")[0]

    ids.append(id_)


In [13]:
print(len(ids))

59


In [15]:
df["id" ] = df["id"].astype(str)

df = df[~df["id"].isin(ids)]

print(df.shape)

(41, 15)


In [16]:
agent = Agent(system_prompt="You are an AI assistant", agent_name="csp_agent")

for i, row in df.iterrows():
    
    id_ = row["id"]
    
    user_query = row["question"]

    response = agent.perform_action(user_query=user_query)


    with open(os.path.join(path, f"gemini_pred_{id_}.txt"), "w") as f:
        f.write(response)

In [4]:
import os
import re

count = 0
path = r"./gemini_responses"
for file in os.listdir(path):
    fp = os.path.join(path, file)

    with open(fp, "r") as f:
        lines = f.readlines()
    
    text = "\n".join(lines)
    if re.search(r"final answer:", text.lower()) is not None:

        reqd_ = text.split("Final Answer:")[1]
        
        if reqd_:
            count += 1

print(count)

14
